In [1]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

C:\Users\DELL\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [4]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [5]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [6]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(17, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = model = Model(inputs=baseModel.input, outputs=fcHead)

In [7]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [8]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [11]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

Epoch 1/25
34/34 [==============================] - 36s 1s/step - loss: 5.8708 - acc: 0.1443 - val_loss: 1.8312 - val_acc: 0.4805
Epoch 2/25
34/34 [==============================] - 25s 727ms/step - loss: 2.2167 - acc: 0.3097 - val_loss: 1.4920 - val_acc: 0.4833
Epoch 3/25
34/34 [==============================] - 21s 631ms/step - loss: 1.8023 - acc: 0.4292 - val_loss: 1.2073 - val_acc: 0.6583
Epoch 4/25
34/34 [==============================] - 21s 632ms/step - loss: 1.5419 - acc: 0.5064 - val_loss: 0.9879 - val_acc: 0.7208
Epoch 5/25
34/34 [==============================] - 22s 634ms/step - loss: 1.4261 - acc: 0.5506 - val_loss: 0.6725 - val_acc: 0.8083
Epoch 6/25
34/34 [==============================] - 22s 633ms/step - loss: 1.3022 - acc: 0.5873 - val_loss: 0.7243 - val_acc: 0.7667
Epoch 7/25
34/34 [==============================] - 22s 636ms/step - loss: 1.1401 - acc: 0.6241 - val_loss: 0.6351 - val_acc: 0.8708
Epoch 8/25
34/34 [==============================] - 22s 636ms/step - los

In [13]:
# unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True

numOfEpoch = 35
opt = SGD(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

Epoch 1/35
34/34 [==============================] - 26s 771ms/step - loss: 0.4242 - acc: 0.8603 - val_loss: 0.3123 - val_acc: 0.9258
Epoch 2/35
34/34 [==============================] - 24s 712ms/step - loss: 0.3769 - acc: 0.8814 - val_loss: 0.2623 - val_acc: 0.9250
Epoch 3/35
34/34 [==============================] - 24s 709ms/step - loss: 0.3101 - acc: 0.9062 - val_loss: 0.2925 - val_acc: 0.9375
Epoch 4/35
34/34 [==============================] - 24s 704ms/step - loss: 0.2902 - acc: 0.9035 - val_loss: 0.4253 - val_acc: 0.9292
Epoch 5/35
34/34 [==============================] - 24s 705ms/step - loss: 0.2829 - acc: 0.8961 - val_loss: 0.3271 - val_acc: 0.9417
Epoch 6/35
34/34 [==============================] - 24s 707ms/step - loss: 0.2556 - acc: 0.9191 - val_loss: 0.1768 - val_acc: 0.9250
Epoch 7/35
34/34 [==============================] - 24s 706ms/step - loss: 0.2351 - acc: 0.9228 - val_loss: 0.3223 - val_acc: 0.9375
Epoch 8/35
34/34 [==============================] - 24s 707ms/step - 